See https://www.mlflow.org/docs/latest/getting-started/quickstart-2/index.html#set-up

In [1]:
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

import mlflow
from mlflow.models import infer_signature

2023-11-18 00:51:33.646375: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
mlflow.set_tracking_uri("http://mlserver:8080")

mlflow.autolog()

mlflow.set_experiment("Wine Tasting")

2023/11/18 00:51:35 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2023/11/18 00:51:35 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/11/18 00:51:35 INFO mlflow.tracking.fluent: Experiment with name 'Wine Tasting' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/814171979374804678', creation_time=1700268695878, experiment_id='814171979374804678', last_update_time=1700268695878, lifecycle_stage='active', name='Wine Tasting', tags={}>

In [3]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

In [4]:
def train_model(params, train_x, train_y, valid_x, valid_y, test_x, test_y, epochs):
    # Define model architecture
    model = Sequential()
    model.add(
        Lambda(lambda x: (x - np.mean(train_x, axis=0)) / np.std(train_x, axis=0))
    )
    model.add(Dense(64, activation="relu", input_shape=(train_x.shape[1],)))
    model.add(Dense(1))

    # Compile model
    model.compile(
        optimizer=SGD(lr=params["lr"], momentum=params["momentum"]),
        loss="mean_squared_error",
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        # Fit model
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            verbose=0,
        )

        # Evaluate the model
        predicted_qualities = model.predict(test_x)
        rmse = np.sqrt(mean_squared_error(test_y, predicted_qualities))

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("rmse", rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": rmse, "status": STATUS_OK, "model": model}


In [5]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
        epochs=32,  # Or any other number of epochs
    )
    return result


In [6]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [7]:
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=12,  # Set to a higher number to explore more hyperparameter configurations
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("rmse", best_run["loss"])
  

  0%|          | 0/12 [00:00<?, ?trial/s, best loss=?]

2023-11-18 00:51:37.236065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-18 00:51:37.244976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-18 00:51:37.245030: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-18 00:51:37.247154: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-18 00:51:37.247249: I tensorflow/compile

1/1 [==============================] - 0s 70ms/step  

  0%|          | 0/12 [00:16<?, ?trial/s, best loss=?]INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(11, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f35d01702e0>, 139869101548736), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(11, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f35d01702e0>, 139869101548736), {}).



INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f35a6d1b010>, 139868830172016), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f35552dbc10>, 139867058124432), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f35552dbc10>, 139867058124432), {}).




INFO:tensorflow:Assets written to: /tmp/tmpsx1kqtsp/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpsx1kqtsp/model/data/model/assets

2023/11/18 00:56:57 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.12.0+nv23.06) contains a local version label (+nv23.6). MLflow logged a pip requirement for this package as 'tensorflow==2.12.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.

2023/11/18 00:57:01 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.12.0+nv23.06) contains a local version label (+nv23.6). MLflow logged a pip requirement for this package as 'tensorflow==2.12.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.



100%|██████████| 12/12 [05:24<00:00, 27.06s/trial, best loss: 0.6926060786762491]


In [8]:
mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(11, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f35552a3760>, 139867038283808), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(11, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f35552a3760>, 139867038283808), {}).



INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3555284400>, 139867038283728), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3555284400>, 139867038283728), {}).



INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 1), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3555286260>, 139867037703168), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 1), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f3555286260>, 139867037703168), {}).



INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f355513d6f0>, 139867037703008), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f355513d6f0>, 139867037703008), {}).




INFO:tensorflow:Assets written to: /tmp/tmp5rdb5bu6/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp5rdb5bu6/model/data/model/assets

2023/11/18 00:57:02 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.12.0+nv23.06) contains a local version label (+nv23.6). MLflow logged a pip requirement for this package as 'tensorflow==2.12.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/11/18 00:57:06 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.12.0+nv23.06) contains a local version label (+nv23.6). MLflow logged a pip requirement for this package as 'tensorflow==2.12.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


In [9]:
# Print out the best parameters and corresponding loss
print(f"Best parameters: {best}")
print(f"Best rmse: {best_run['loss']}")

Best parameters: {'lr': 0.000116097629311053, 'momentum': 0.5195066447486464}
Best rmse: 0.6926060786762491


In [10]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7
